# Preface
The goal is to create a recommendation system for MTG Players to be able to use. 

~~~
response = requests.get('https://api.scryfall.com/bulk-data') #to get all data API endpoints
~~~
~~~
    import json
        def jprint(obj):
            text = json.dumps(obj, sort_keys = True, indent = 4)
            print(text)
    
            jprint(response.json())
~~~

# Observations
* Brought in both oracle and card datasets. upon inspection, the oracle will be best to go with. Less repeated cards, easier to work with, unique id_identifiers

Ratio of missing values = the number of missing values / total number of observations * 100


* Dropped for modeling:
    * mtgo_foil_id is not needed.
    * Flavor text is not needed. We can add a line about where to find all flavor text, or import into database at later time for search recommendations
    * security_stamp is not needed.
    * preview is not needed for modeling purposes
    * arena_id could be useful for an online rec system
    * watermark not needed
    * produced mana can be removed. If not, we need to replace NaN values with just Not Applicable
    * all parts could be viewed as a target for combo model
    * object not needed. every object is a card-type object
    * lang contains only 10 japanese cards out of 26000. Removed column.
    * type can help fix color column. We can use type to tell if it's an artifact, then create a colorless condition
    * mana_cost: could be fixed. It has a few cards that are duplicates. Dropped those. Then there are leftovers that are dual faced cards. Costs are different for each. We could fill these with cmc costs instead or look at cards individually for this issue. For now, dropped column.
* Fixed:
    * power: set na to 0. Missing value indicates a card that does not have a power value(i.e. enchants, sorcery, instant)
    * toughness: set na to 0. Missing value indicates a card that does not have a toughness value(i.e. enchants, sorcery, instant)
    * edhrec_rank: filled na values using the max rec number +1 and setting an incremental counter, which, for each na value, added 1 to the counter and saved it as that value. This does not accurately reflect the ranking of the cards, but allows us to at least get an understanding to the rest of the ranked cards. 
    * mana_cost: removed from a dictionary and seperated values with a /. Replaced U with BL for blue.
    * keywords: removed from list, replaced na values with None, to allow our program to understand that there is no keywords associated with that card.
    * colors: removed from list, filled na values with C for colorless. Meaning that the card is cast with colorless mana only.
    * color_identity: removed from list, filled na values with C for colorless. Meaning that the card is cast with colorless mana only.
    * oracle_id: identified as a unique ID. set to index to allow unique key.
    * cmc: cmc had a single card listed with a 0.5 mana value. Replaced with 1. converted column to integer after removing single float value from column.
    

In [5]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [57]:
class Data_Handling(object):
    def wrangle(filepath):
        """
        Wrangles in filepath. Tells the difference between JSON and CSV and creates dataframe.
    
        NA Values are handled as:
    
        edhrec_rank: replaces na values with incremental counter from the last recommended value to the length of the dataframe.
        power and toughness: na values represent cards that are non creature. Replaced with a zero.
        """
        if filepath.endswith(".json"):
            df = pd.read_json(filepath)
        else:
            df = pd.read_csv(filepath)
        
        #Fix NA Values for edhrec_rank
        if 'edhrec_rank' in df.columns:
            edh_fix = df[df['edhrec_rank'].isna() == True]
            counter = 22665 # Max rank + 1
    
            edh_fix.edhrec_rank = range(counter, (counter + len(edh_fix)))
            df.loc[edh_fix.index, :] = edh_fix[:]
            df['edhrec_rank'] = df['edhrec_rank'].astype('int64')
    
        # Fix power column
        if 'power' in df.columns:
            df['power'].loc[df['power'].isna() == True] = 0
        
        # Fix Toughness Columns
        if 'toughness' in df.columns:
            df['toughness'].loc[df['toughness'].isna() == True] = 0
            
        # Fix CMC Column
        if 'cmc' in df.columns:
            df['cmc'].loc[17411] = 1
            df['cmc'] = df['cmc'].astype('int64')
        
        if 'oracle_id' in df.columns:
            df.set_index('oracle_id', inplace=True)
        
        if 'colors' in df.columns:
            df['colors'] = df['colors'].str[0]
            df['colors'] = df['colors'].str.replace('U', 'BL')
            df['colors'].fillna('C', inplace=True)

        if 'color_identity' in df.columns:
            df['color_identity'] = df['color_identity'].str[0]
            df['color_identity'] = df['color_identity'].str.replace('U', 'BL')
            df['color_identity'].fillna('C', inplace=True)
            
        if 'keywords' in df.columns:
            df['keywords'] = df['keywords'].str[0]
            df['keywords'].fillna('None', inplace =True)
        
        if 'mana_cost' in df.columns:
            df['mana_cost'].fillna(df['cmc'].astype(str), inplace=True)
            df['mana_cost'] = df['mana_cost'].str.replace('U', 'BL')
            l = []
            for val in df.mana_cost:
                val = re.sub(r'[{]', '', str(val))
                val = re.sub(r'[}]', '/', val)
                val = val.strip('/')
                l.append(val)
            df['mana_cost'] = l
        
            
        return df
    
    def drop_cols(df):
        # Drops all columns with greater than 35% NA values
        #Drops mtgo_id column, which has a high number of NA values as well
        drop_cols = [col for col in df.columns if (df[col].isna().sum() / len(df) *100) > 35]
        drop_cols.append('mtgo_id')
        df.drop(columns = drop_cols, inplace=True)
        
        return df
    
    def modeling_prep_mtg_oracle(df):
        # Drop columns for modeling purposes
        drop_cols = ['id', 'multiverse_ids', 'tcgplayer_id', 'cardmarket_id', 'lang', 'object', 
                     'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 
                     'image_uris', 'games', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'prices',
                     'legalities', 'reserved', 'foil', 'nonfoil', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 
                     'border_color', 'oversized', 'finishes', 'scryfall_set_uri', 'rulings_uri', 'promo', 'set', 'set_uri', 'set_search_uri', 
                     'reprint', 'variation', 'set_id', 'prints_search_uri', 'collector_number', 'digital']
        if 'related_uris' in df.columns:
            target_cards = df['related_uris']
            drop_cols.append('related_uris')
        df.drop(columns = drop_cols, inplace= True)
        
        if 'type_line' in df.columns:
            a = df[df['type_line'].str.contains('Token Creature')]
            df.drop(labels=a.index, inplace=True)
                
        if 'set_name' in df.columns:
            c = df[df['set_name'].str.contains('Art Series')]
            df.drop(labels = c.index, inplace=True)

        return df, target_cards

In [58]:
df = Data_Handling.wrangle('https://c2.scryfall.com/file/scryfall-bulk/oracle-cards/oracle-cards-20220412211242.json')
df, target_cards = Data_Handling.modeling_prep_mtg_oracle(df)
df = Data_Handling.drop_cols(df)
df.head()

,name,mana_cost,cmc,type_line,oracle_text,colors,color_identity,keywords,set_name,set_type,rarity,edhrec_rank,power,toughness
oracle_id,,,,,,,,,,,,,,
0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,3,3,Artifact,"As long as Static Orb is untapped, players can...",C,C,None,Seventh Edition,core,rare,2631,0,0
0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,BL,1,Enchantment — Aura,Enchant creature\nEnchanted creature gets -3/-0.,BL,BL,Enchant,Magic 2014,core,common,21574,0,0
0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,G/G,2,Sorcery,Choose one —\n• Return target permanent card f...,G,G,Entwine,Commander 2019,commander,rare,4080,0,0
000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,1/BL,2,Creature — Bird,Flying (This creature can't be blocked except ...,BL,BL,Flying,Ninth Edition,core,common,12416,1,2
000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,1/BL,2,Creature — Sponge,{T}: Target creature loses your choice of flyi...,BL,BL,None,Urza's Legacy,expansion,uncommon,18864,1,1


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24252 entries, 0004ebd0-dfd6-4276-b4a6-de0003e94237 to ffff90c3-63c4-4dee-a21d-6b2b113f4f80
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            24252 non-null  object
 1   mana_cost       24252 non-null  object
 2   cmc             24252 non-null  int64 
 3   type_line       24252 non-null  object
 4   oracle_text     23789 non-null  object
 5   colors          24252 non-null  object
 6   color_identity  24252 non-null  object
 7   keywords        24252 non-null  object
 8   set_name        24252 non-null  object
 9   set_type        24252 non-null  object
 10  rarity          24252 non-null  object
 11  edhrec_rank     24252 non-null  int64 
 12  power           24252 non-null  object
 13  toughness       24252 non-null  object
dtypes: int64(2), object(12)
memory usage: 2.8+ MB


In [66]:
df[df['oracle_text'].isna() == True]
# Look at other .json files for oracle text on single cards. Link URI's for both in the line.

,name,mana_cost,cmc,type_line,oracle_text,colors,color_identity,keywords,set_name,set_type,rarity,edhrec_rank,power,toughness
oracle_id,,,,,,,,,,,,,,
01ea243f-12f5-4315-9523-9c78fce5a053,Foraging Squirrels // Foraging Squirrels (cont'd),0,0,Card // Card,NaN,C,C,None,Modern Horizons 2 Minigames,funny,common,22683,0,0
0272ca81-e727-4f4b-b06e-072d70bb5558,Ulvenwald Captive // Ulvenwald Abomination,2,2,Creature — Werewolf Horror // Creature — Eldra...,NaN,C,G,Transform,Eldritch Moon,expansion,common,6247,0,0
02c60b8b-f665-4279-a00f-ae03991df4a6,Azusa's Many Journeys // Likeness of the Seeker,2,2,Enchantment — Saga // Enchantment Creature — H...,NaN,C,G,Transform,Kamigawa: Neon Dynasty,expansion,uncommon,7919,0,0
03d3fee8-f7de-4152-8c18-6d279cc6c125,Heaven // Earth,X/G/ // X/R/R,3,Instant // Sorcery,NaN,G,G,Aftermath,Amonkhet,expansion,rare,9346,0,0
043370fd-9cfe-47ce-8019-e915cee1ae95,Breaking // Entering,BL/B/ // 4/B/R,8,Sorcery // Sorcery,NaN,B,B,Mill,Dragon's Maze,expansion,rare,12558,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff0ab867-b710-4b1a-baed-95fc3cf68f79,Valakut Awakening // Valakut Stoneforge,3,3,Instant // Land,NaN,C,R,None,Zendikar Rising,expansion,rare,20450,0,0
ff7c12dd-1a1a-417a-b08a-d1346430858e,Bind // Liberate,1/G/ // 1/W,2,Instant // Instant,NaN,G,G,None,Mystery Booster Playtest Cards 2021,funny,rare,25455,0,0
ffc1fe11-0ecb-4497-a266-0817c79477e9,Vengeful Strangler // Strangling Grasp,2,2,Creature — Human Rogue // Enchantment — Aura,NaN,C,B,Transform,Innistrad: Midnight Hunt,expansion,uncommon,11521,0,0
